# Lecture 24           
                                            
## Forecasting a time-series object         
   - Data munging with time-series (ts)
   - Sample splitting with ts          
   - ARIMA models                      
     - self specified                  
     - auto ARIMA                      
     - adding trend and seasonality    
   - Cross-validation with ARIMA       
   - Vector Autoregressive Model (VAR) 
     - estimation and cv               
   - Forecasting                       
     - comparing model based on        
       forecasting performance         
     - external validity check         
     - Fan-chart: assessing risks            

Case-studies:

   - CH18B Forecasting a home price index case-schiller-la dataset  
                                             
Data used:

    case-schiller-la                 

___

In [ ]:
import os
from pathlib import Path
import pandas as pd
import numpy as np
from mizani.breaks import date_breaks
from mizani.formatters import date_format
from plotnine import *
from datetime import datetime
import sys
import statsmodels
import patsy
from pmdarima.arima import auto_arima
import statsmodels.formula.api as smf
import warnings
from sklearn.metrics import mean_squared_error

warnings.filterwarnings("ignore")

### Data preparation

In [ ]:
data = pd.read_csv("https://osf.io/8r7na/download")

In [ ]:
data.head()

- 18 years data
- 1 year holdout
- 4 years of test
- 13 years of train (rolling window)

Pick if seasonal or non seasonal version used, will be cut later here we pick pn, not seasonally adjuste

Create date format

In [ ]:
data = data.assign(date=lambda x: x.date.str[0:7])


Create newly named variable -> seasonally not adjusted

In [ ]:
data = data.rename({"pn": "p", "us": "u", "emps": "emp"}, axis=1)

 Create logs, lags, differences and trend + monthly dummmies

In [ ]:
data = data.sort_values(by=["date"]).assign(
    dp=lambda x: x.p.diff(1),
    p_lag=lambda x: x.p.shift(1),
    lnp=lambda x: np.log(x.p),
    dlnp=lambda x: x.lnp.diff(1),
    lnp_lag=lambda x: x.lnp.shift(1),
    dlnp_lag=lambda x: x.dlnp.shift(1),
    du=lambda x: x.u.diff(1),
    lnemp=lambda x: np.log(x.emp),
    dlnemp=lambda x: x.lnemp.diff(1),
    trend=lambda x: range(1, data.shape[0] + 1),
)

Save as time-series tibble

In [ ]:
data_all = data.copy()

 Focus on dates from 2000 through 2017


In [ ]:
data = data.loc[lambda x: x.year <= 2017]

### EXPLORE our variables

Plot price index

In [ ]:
limits = datetime(2000, 1, 1), datetime(2018, 1, 1)
breaks = date_breaks("1 year")

price_index_plot = (
    ggplot(data, aes(x="date", y="p", group=1))
    + geom_line(size=1)
    + scale_y_continuous(limits=[50, 300], breaks=np.arange(50, 301, 50))
    + scale_x_date(breaks=breaks(limits)[::3], labels=date_format("%b%Y"))
    + labs(y="Case-shiller Price index", x="Date (month)")
    + theme_bw()
)
price_index_plot

Plot difference of price index

In [ ]:
dp_plot = (
    ggplot(data, aes(x="date", y="dp", group=1))
    + geom_line(size=1)
    + scale_y_continuous(limits=[-10, 8], breaks=np.arange(-10, 9, 2))
    + scale_x_date(breaks=breaks(limits)[::3], labels=date_format("%b%Y"))
    + labs(y="First difference of price index", x="Date (month)")
    + theme_bw()
)
dp_plot

Plot log difference of price index (-> ~percentage change)

In [ ]:
limits = datetime(2000, 1, 1), datetime(2018, 1, 1)
breaks = date_breaks("1 year")

dlnp_plot = (
    ggplot(data, aes(x="date", y="dlnp", group=1))
    + geom_line(size=1)
    + scale_y_continuous(limits=[-0.04, 0.04], breaks=np.arange(-0.04, 0.05, 0.01))
    + scale_x_date(breaks=breaks(limits)[::3], labels=date_format("%b%Y"))
    + labs(y="Log first difference of price index", x="Date (month)")
    + theme_bw()
)
dlnp_plot

Plot employment

In [ ]:
emp_plot = (
    ggplot(data, aes(x="date", y="emp", group=1))
    + geom_line(size=1)
    + scale_x_date(breaks=breaks(limits)[::3], labels=date_format("%b%Y"))
    + labs(y="Employment (in thousands)", x="Date (month)")
    + theme_bw()
)
emp_plot

Plot log diff employment

In [ ]:
lnemp_plot = (
    ggplot(data, aes(x="date", y="dlnemp", group=1))
    + geom_line(size=1)
    + scale_x_date(breaks=breaks(limits)[::3], labels=date_format("%b%Y"))
    + labs(y="Log change in employment", x="Date (month)")
    + theme_bw()
)

lnemp_plot

Plot unemployment rate

In [ ]:
u_plot = (
    ggplot(data, aes(x="date", y="u", group=1))
    + geom_line(size=1)
    + scale_x_date(breaks=breaks(limits)[::3], labels=date_format("%b%Y"))
    + labs(y="Unemployment rate (percent)", x="Date (month)")
    + theme_bw()
)

u_plot

Plot diff unemployment

In [ ]:
du_plot = (
    ggplot(data, aes(x="date", y="du", group=1))
    + geom_line(size=1)
    + scale_x_date(breaks=breaks(limits)[::3], labels=date_format("%b%Y"))
    + labs(y="Change in unemployment rate", x="Date (month)")
    + theme_bw()
)

du_plot

### Create work set and holdout set

In [ ]:
data["date"] = data["date"].apply(lambda x: datetime.strptime(x, "%Y-%m"))

Create work and holdout sets

In [ ]:
data_holdout = data.loc[lambda x: x["year"] == 2017, :]
data_work = data.loc[lambda x: x["year"] < 2017, :]

Prepare for cross-validation, define size of train -> rolling window type (cv=4)

In [ ]:
for year in range(2013, 2017):
    fold = year - 2012
    data_work["test" + str(fold)] = data_work["year"] == year
    data_work["train" + str(fold)] = (data_work["year"] <= year - 1) & (
        data_work["year"] >= year - 13
    )

 Use tseries of price index only
 
Fit  models with months, trend, ARIMA

In [ ]:
from statsmodels.tsa.arima.model import ARIMA
from statsmodels.tsa.api import VAR

Model M1 – p on trend & seasonality

In [ ]:
mse_1 = []
for i in range(1, 5):

    train_data = data_work.loc[lambda x: x["train" + str(i)] == 1]
    test_data = data_work.loc[lambda x: x["test" + str(i)] == 1]

    model1 = smf.ols("p ~ trend + C(month)", train_data).fit()

    phat = model1.predict(test_data)

    errsq = np.square(test_data.p.values - phat)

    mse_1.append(np.mean(errsq))

rmse_cv_m1 = np.sqrt(np.mean(mse_1))

Model M2 – p ARIMA(1,1,2)
 
 Get order from auto_arima

In [ ]:
auto_arima_m2 = auto_arima(
    y=data_work.p,
    start_p=0,
    max_p=1,  # without this constrain, python returns other ARIMA order
    max_order=0,
    seasonal=False,
)

mse_2 = []
for i in range(1, 5):

    train_data = data_work.loc[lambda x: x["train" + str(i)] == 1]
    test_data = data_work.loc[lambda x: x["test" + str(i)] == 1]

    model2 = ARIMA(train_data.p, order=auto_arima_m2.get_params()["order"]).fit()

    phat = model2.forecast(steps=12)

    errsq = np.square(test_data.p.values - phat)

    mse_2.append(np.mean(errsq))

rmse_cv_m2 = np.sqrt(np.mean(mse_2))

Model M3 – p ARIMA(1,1,0)

Get order from auto_arima

In [ ]:
auto_arima_m3 = auto_arima(
    y=data_work.p, X=pd.get_dummies(data_work.month), max_order=0, seasonal=False
)

mse_3 = []
for i in range(1, 5):

    train_data = data_work.loc[lambda x: x["train" + str(i)] == 1]
    test_data = data_work.loc[lambda x: x["test" + str(i)] == 1]

    model3 = ARIMA(
        train_data.p,
        exog=pd.get_dummies(train_data.month),
        order=auto_arima_m3.get_params()["order"],
    ).fit()

    phat = model3.forecast(steps=12, exog=pd.get_dummies(test_data.month))

    errsq = np.square(test_data.p.values - phat)

    mse_3.append(np.mean(errsq))

rmse_cv_m3 = np.sqrt(np.mean(mse_3))

Model M4 –  p ARIMA(2,0,0) + seasonality + trend

Get order from auto_arima

In [ ]:
auto_arima_m4 = auto_arima(
    y=data_work.p,
    X=pd.get_dummies(data_work.month).assign(t=data_work.trend),
    max_order=0,
    seasonal=False,
)

mse_4 = []
for i in range(1, 5):

    train_data = data_work.loc[lambda x: x["train" + str(i)] == 1]
    test_data = data_work.loc[lambda x: x["test" + str(i)] == 1]

    model4 = ARIMA(
        train_data.p,
        exog=pd.get_dummies(train_data.month),
        trend="t",
        order=auto_arima_m4.get_params()["order"],
    ).fit()

    phat = model4.forecast(steps=12, exog=pd.get_dummies(test_data.month), trend="t")

    errsq = np.square(test_data.p.values - phat)

    mse_4.append(np.mean(errsq))

rmse_cv_m4 = np.sqrt(np.mean(mse_4))

Model M5 – dp ~ month + trend, without any ARIMA


In [ ]:
mse_5 = []
for i in range(1, 5):

    train_data = data_work.loc[lambda x: x["train" + str(i)] == 1]
    test_data = data_work.loc[lambda x: x["test" + str(i)] == 1]

    model5 = smf.ols("dp ~ trend + C(month)", train_data).fit()

    dphat = model5.predict(test_data)

    test_data["phat"] = None
    for i in range(0, 12):
        if i == 0:
            test_data.iloc[i, -1] = train_data["p"].values[-1] + dphat.iloc[i]
        else:
            test_data.iloc[i, -1] = test_data.iloc[i - 1, -1] + dphat.iloc[i]

    errsq = np.square(test_data["p"] - test_data["phat"])

    mse_5.append(np.mean(errsq))

rmse_cv_m5 = np.sqrt(np.mean(mse_5))

Model M6 – lnp ARIMA(0,2,0) + seasonality

 Get order from auto_arima

In [ ]:
auto_arima_m6 = auto_arima(
    y=data_work.lnp,
    X=pd.get_dummies(data_work.month),
    d=2,  # without this constrain, python returns other ARIMA order
    max_order=0,
    seasonal=True,
)

mse_6 = []
for i in range(1, 5):

    train_data = data_work.loc[lambda x: x["train" + str(i)] == 1]
    test_data = data_work.loc[lambda x: x["test" + str(i)] == 1]

    model6 = ARIMA(
        train_data.lnp,
        exog=pd.get_dummies(train_data.month),
        order=auto_arima_m6.get_params()["order"],
    ).fit()

    lnphat = model6.forecast(steps=12, exog=pd.get_dummies(test_data.month))

    corrb = mean_squared_error(test_data.lnp, lnphat)

    phat = np.exp((lnphat + corrb / 2))

    errsq = np.square(test_data.p.values - phat)

    mse_6.append(np.mean(errsq))

rmse_cv_m6 = np.sqrt(np.mean(mse_6))

VAR

In [ ]:
mse_var = []
for i in range(1, 5):

    train_data = data_work.loc[lambda x: x["train" + str(i)] == 1, :].dropna()
    test_data = data_work.loc[lambda x: x["test" + str(i)] == 1, :].dropna()

    model7 = VAR(train_data[["dp", "du", "dlnemp"]]).fit(1)

    dphat = model7.forecast(
        train_data[["dp", "du", "dlnemp"]].values[-model7.k_ar :], steps=12
    )[:, 0]

    test_data["phat"] = None
    for i in range(0, 12):
        if i == 0:
            test_data.iloc[i, -1] = train_data["p"].values[-1] + dphat[i]
        else:
            test_data.iloc[i, -1] = test_data.iloc[i - 1, -1] + dphat[i]

    errsq = np.square(test_data["p"] - test_data["phat"])

    mse_var.append(np.mean(errsq))

rmse_cv_m7 = np.sqrt(np.mean(mse_var))

**Note** This VAR includes month dummies. Textbook results are from R, where we could not figure out how to inlcude month dummies


In [ ]:
mse_var_season = []
for i in range(1, 5):

    train_data = data_work.loc[lambda x: x["train" + str(i)] == 1, :].dropna()
    test_data = data_work.loc[lambda x: x["test" + str(i)] == 1, :].dropna()

    model7_season = VAR(
        train_data[["dp", "du", "dlnemp"]], exog=pd.get_dummies(train_data["month"])
    ).fit(1)

    dphat = model7_season.forecast(
        train_data[["dp", "du", "dlnemp"]].values[-model7_season.k_ar :],
        exog_future=pd.get_dummies(test_data["month"]).values,
        steps=12,
    )[:, 0]

    test_data["phat"] = None
    for i in range(0, 12):
        if i == 0:
            test_data.iloc[i, -1] = train_data["p"].values[-1] + dphat[i]
        else:
            test_data.iloc[i, -1] = test_data.iloc[i - 1, -1] + dphat[i]

    errsq = np.square(test_data["p"] - test_data["phat"])

    mse_var_season.append(np.mean(errsq))

rmse_cv_m7_season = np.sqrt(np.mean(mse_var_season))

In [ ]:
pd.DataFrame(
    [mse_1, mse_2, mse_3, mse_4, mse_5, mse_6, mse_var],
    columns=["Fold" + str(i) for i in range(1, 5)],
).apply(np.sqrt).assign(
    Average=[
        rmse_cv_m1,
        rmse_cv_m2,
        rmse_cv_m3,
        rmse_cv_m4,
        rmse_cv_m5,
        rmse_cv_m6,
        rmse_cv_m7,
    ],
    model=["M" + str(i) for i in range(1, 7)] + ["M7 (var)"],
).round(
    2
).set_index(
    "model"
)

In [ ]:
print("RMSE for VAR with seasonality (not in book):", rmse_cv_m7_season)

### predict for holdout

Best model is M4 – re-estimate best models on full work set

In [ ]:
model_final = ARIMA(
    data_work.p, exog=pd.get_dummies(data_work.month), trend="t", order=(2, 0, 0)
).fit()

In [ ]:
pred_final = model_final.get_forecast(
    steps=12, exog=pd.get_dummies(data_holdout.month), trend="t"
)

In [ ]:
forecast_holdout_best = (
    data_holdout.assign(p_pred=pred_final.predicted_mean.values, model="best")
    .join(pred_final.conf_int(alpha=0.2))
    .filter(["model", "p_pred", "lower p", "upper p"])
)

In [ ]:
data_plot = data.join(forecast_holdout_best).loc[lambda x: x.year >= 2015]

In [ ]:
data_plot_1 = pd.melt(data_plot.filter(["date","p","p_pred"]),["date"])

pred_p_plot = (
    ggplot(data_plot_1, aes(x="date", y="value", color="variable"))
    + geom_line(size=1)
    + ylab("Case-Shiller Home Price Index")
    + xlab("Date (month)")
    + scale_color_manual(
        name=" ", values=("red","blue"), labels=("Actual", "Prediction")
    )
    + scale_x_date(date_breaks="1 years", labels=date_format("%b%Y"))
    + theme_bw()
    + theme(legend_position=(0.7, 0.3), legend_direction="horizontal")
)
pred_p_plot

In [ ]:
pred_p_plot = (
    ggplot(data_plot.reset_index(), aes(x="date"))
    + geom_line(aes(y="p"), color="red", size=0.7)
    + geom_line(aes(y="p_pred"), color="blue", size=1)
    + geom_line(aes(y="lower p"), color="blue", size=0)
    + geom_line(aes(y="upper p"), color="blue", size=0)
    + geom_ribbon(aes(ymin="lower p", ymax="upper p"), fill="blue", alpha=0.3)
    + ylab("Case-Shiller Home Price Index")
    + xlab("Date (month)")
    + theme_bw()
    + scale_x_date(date_breaks="1 years", labels=date_format("%b%Y"))
)
pred_p_plot

In [ ]:
errsq = np.square(data_holdout.p.values - forecast_holdout_best.p_pred)

rmse_holdout = np.mean(errsq)
rmse_holdout

### EXTERNAL VALIDITY
### do the prediction for an extra year

In [ ]:
data = data_all.copy()
data["date"] = data["date"].apply(lambda x: datetime.strptime(x, "%Y-%m"))

Create work and holdout sets

In [ ]:
data_holdout = data.loc[lambda x: x["year"] == 2018, :]
data_work = data.loc[lambda x: x["year"] < 2018, :]

Best model is M4 – re-estimate best models on full work set

In [ ]:
model_final = ARIMA(
    data_work.p, exog=pd.get_dummies(data_work.month), trend="t", order=(2, 0, 0)
).fit()

In [ ]:
pred_final = model_final.get_forecast(
    steps=12, exog=pd.get_dummies(data_holdout.month), trend="t"
)

In [ ]:
forecast_holdout_best = (
    data_holdout.assign(p_pred=pred_final.predicted_mean.values, model="best")
    .join(pred_final.conf_int(alpha=0.2))
    .filter(["model", "p_pred", "lower p", "upper p"])
)

In [ ]:
data_plot = data.join(forecast_holdout_best).loc[lambda x: x.year >= 2015]

In [ ]:
data_plot_1 = pd.melt(data_plot.filter(["date","p","p_pred"]),["date"])

pred_p_plot = (
    ggplot(data_plot_1, aes(x="date", y="value", color="variable"))
    + geom_line(size=1)
    + ylab("Case-Shiller Home Price Index")
    + xlab("Date (month)")
    + scale_color_manual(
        name=" ", values=("red","blue"), labels=("Actual", "Prediction")
    )
    + scale_x_date(date_breaks="1 years", labels=date_format("%b%Y"))
    + theme_bw()
    + theme(legend_position=(0.7, 0.3), legend_direction="horizontal")
)
pred_p_plot

with uncertainty fan

In [ ]:
pred_p_plot = (
    ggplot(data_plot.reset_index(), aes(x="date"))
    + geom_line(aes(y="p"), color="red", size=0.7)
    + geom_line(aes(y="p_pred"), color="blue", size=1.3)
    + geom_line(aes(y="lower p"), color="blue", size=0)
    + geom_line(aes(y="upper p"), color="blue", size=0)
    + geom_ribbon(aes(ymin="lower p", ymax="upper p"), fill="blue", alpha=0.3)
    + ylab("Case-Shiller Home Price Index")
    + xlab("Date (month)")
    + theme_bw()
    + scale_x_date(date_breaks="1 years", labels=date_format("%b%Y"))
)
pred_p_plot

In [ ]:
errsq = np.square(data_holdout.p.values - forecast_holdout_best.p_pred)

rmse_holdout = np.mean(errsq)
rmse_holdout